
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [3]:
# the path to your HDF5 formatted snps file
data = "./M_ru_3rm_v9.snps.hdf5"

In [4]:
imap = {
#"ref": ["reference"],
"Inam": ['M_ru_A7875_In','M_ru_T14456_In', 'M_ru_T23245_In', 'M_ru_T23246_In', 'M_ru_T23416_In', 'M_ru_T23478_In'],
"Puru": ['M_ru_A10311_pu','M_ru_A10329_pu', 'M_ru_A1380_pu', 'M_ru_A2741_pu', 'M_ru_A436_pu', 'M_ru_A440_pu'],
"JiGu": ['M_ru_A207_jigu', 'M_ru_T3228_jigu', 'M_ru_T7634_jigu'],
"Mach": ['M_ru_T13253_ma','M_ru_A474_ma', 'M_ru_T3164_ma', 'M_ru_J265_ma', ],#'M_ru_T368_ma', 'M_ru_T381_ma', 'M_ru_T476_ma', 'M_ru_T494_ma'
#"Roar": ['M_ru_J640_roar','M_ru_J676_roar'],
#"ArSu": ['M_ru_80819_arsu', 'M_ru_81347_arsu', 'M_ru_85426_arsu'],
#"SuTa": [ 'M_ru_85919_suta','M_ru_77750_suta', 'M_ru_78182_suta', 'M_ru_A11834_suta', 'M_ru_A15176_suta', 'M_ru_A5487_suta', 'M_ru_T10184_suta', 'M_ru_T11780_suta', 'M_ru_T14532_suta', 'M_ru_T14622_suta', 'M_ru_T753_suta', 'M_ru_A9903_pa'],
#"Para": ['M_ru_A16195_pa','M_ru_A9235_pa', 'M_ru_T11079_pa', 'M_ru_T11238_pa', 'M_ru_T12541_pa', 'M_ru_T1649_pa', 'M_ru_T16553_pa', 'M_ru_T19782_pa', 'M_ru_T6500_pa', 'M_ru_T6577_pa']
}
 
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="M_ru_west_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 753598
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_west_str-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_west_str-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/M_ru_3rm_v9_outfiles/analysis-structure/tmp-M_ru_west_str-<built-in function abs>-1.strfile.txt')

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 5:15:22 | running 70 structure jobs 


In [34]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-12092.06,302.369,0.00,0.00
3,10,2.073,-11355.78,794.530,736.28,1647.14
4,10,0.558,-12266.64,2263.758,-910.86,1262.77
5,10,17.907,-11914.73,454.402,351.91,8136.79
6,10,0.055,-19699.61,22816.662,-7784.88,1245.21
7,10,0.955,-28729.70,21746.797,-9030.09,20763.24
8,10,0.000,-16996.55,11170.303,11733.15,0.00


In [35]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="ta1802cefd10c4cd180384a5f2b69a243" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 10000 15000 20000 25000 30000 estLnProbMean 0 10 20 deltaK

In [36]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [37]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [38]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t4fe41fd68e3948018b76ea6a3563e72d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_T13253_ma M_ru_A474_ma M_ru_T3164_ma M_ru_J265_ma M_ru_A7875_In M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A207_jigu M_ru_T3228_jigu M_ru_T7634_jigu 0.0 0.5 1.0

In [39]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [40]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [41]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t7035ee4fac5d4b459b424c3e60aaeaed" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_T13253_ma M_ru_A474_ma M_ru_T3164_ma M_ru_J265_ma M_ru_A7875_In M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A207_jigu M_ru_T3228_jigu M_ru_T7634_jigu 0.0 0.5 1.0

In [42]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [43]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [44]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t89f1b24fadab46af8df5be7bb84017d3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_T13253_ma M_ru_A474_ma M_ru_T3164_ma M_ru_J265_ma M_ru_A7875_In M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A207_jigu M_ru_T3228_jigu M_ru_T7634_jigu 0.0 0.5 1.0

In [45]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [46]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [47]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t3457be6349cb4776a01fb4080eb12cba" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_T13253_ma M_ru_A474_ma M_ru_T3164_ma M_ru_J265_ma M_ru_A7875_In M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A207_jigu M_ru_T3228_jigu M_ru_T7634_jigu 0.0 0.5 1.0

In [48]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [49]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [50]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t58cb61ec63e6408ca6d40de517a1e0fe" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_T13253_ma M_ru_A474_ma M_ru_T3164_ma M_ru_J265_ma M_ru_A7875_In M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A207_jigu M_ru_T3228_jigu M_ru_T7634_jigu 0.0 0.5 1.0

# PCA

In [51]:
#!conda install scikit-learn -c conda-forge -y

In [52]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [53]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 505328
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [54]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 8454/65593


In [55]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b1aa3cae190>,
 <toyplot.mark.Point at 0x2b1aa3cd9690>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8b1d80973ab746bcb63a8f47a1764394" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -8 0 8 16 PC0 (34.3%) explained -10 -5 0 5 10 PC1 (14.1%) explained Inam JiGu Puru Mach

In [56]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("M_ru_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
M_ru_A10311_pu,-6.51,7.64,0.27,-1.82,-1.00,-1.61,7.11,-2.18,-0.11,6.78
M_ru_A10329_pu,-7.79,8.58,-0.34,2.82,-7.30,3.94,-1.17,-6.43,2.41,-2.11
M_ru_A1380_pu,-6.48,8.78,-0.68,-0.05,7.40,1.63,-3.39,-4.26,-2.13,-2.87
M_ru_A207_jigu,-7.40,-7.79,-8.95,0.01,2.08,-0.15,2.58,-0.82,-0.88,2.75
M_ru_A2741_pu,-6.68,7.25,-0.70,-1.95,0.64,-2.07,0.78,3.70,-4.64,-4.60
M_ru_A436_pu,-6.75,8.36,-0.09,-0.63,-5.55,-2.43,-0.97,6.90,-0.33,-0.23
M_ru_A440_pu,-7.21,7.53,-0.72,2.27,5.96,-0.43,-1.95,2.71,3.84,3.24
M_ru_A474_ma,-5.12,-6.08,8.95,-6.76,-0.22,-5.34,0.13,-4.40,-1.66,-1.46
M_ru_A7875_In,13.29,0.28,-0.28,-0.86,0.39,2.97,1.87,-0.34,-0.86,1.80
M_ru_J265_ma,-5.20,-6.84,9.33,7.73,0.51,3.42,3.26,1.46,-5.52,-0.25


# Subsampling with replication

In [57]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 8454/65593


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tcebccdc1e99f4a2f8eabbcfa7384902d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -10 0 10 20 PC0 (33.9%) explained -10 -5 0 5 10 PC1 (14.1%) explained Inam JiGu Puru Mach

# Kmeans imputation (integer)

In [58]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Subsampling SNPs: 8454/65593


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t593e2e619e394062a38365752b45dc01" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -10 0 10 20 PC0 (34.2%) explained -10 0 10 PC1 (13.6%) explained Inam JiGu Puru Mach

In [59]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "M_ru_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [60]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [61]:
pca.run_tsne(subsample=True, perplexity=3.0, n_iter=100000, seed=623)

Subsampling SNPs: 8454/65593


In [62]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t1e1294fe169f4b1cbc2c34665ae573a3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> M_ru_A10311_pu M_ru_A10329_pu M_ru_A1380_pu M_ru_A207_jigu M_ru_A2741_pu M_ru_A436_pu M_ru_A440_pu M_ru_A474_ma M_ru_A7875_In M_ru_J265_ma M_ru_T13253_ma M_ru_T14456_In M_ru_T23245_In M_ru_T23246_In M_ru_T23416_In M_ru_T23478_In M_ru_T3164_ma M_ru_T3228_jigu M_ru_T7634_jigu -200 -100 0 100 200 TNSE component 1 -300 -200 -100 0 TNSE component 2 Inam JiGu Puru Mach

In [63]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "M_ru_TSNE_perp3.pdf")

# GENETIC DISTNANCES

In [64]:
import ipyrad.analysis as ipa
import toyplot

In [65]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 19
Sites before filtering: 819934
Filtered (indels): 0
Filtered (bi-allel): 6174
Filtered (mincov): 505328
Filtered (minmap): 753598
Filtered (combined): 754341
Sites after filtering: 65593
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [66]:
# save to a CSV file
dist.dists.to_csv("M_ru_distances.csv")

In [67]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "M_ru_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [68]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [69]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t6d117bd79f1644968e07f143b1be96f8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.043526 0.042992 0.043694 0.088531 0.089552 0.087509 0.086457 0.086884 0.089003 0.063406 0.063970 0.064046 0.063025 0.065388 0.064550 0.052963 0.051637 0.050905 M_ru_T13253_ma 1 0.043526 0.000000 0.040690 0.041270 0.089552 0.089689 0.088439 0.087540 0.088180 0.089263 0.068422 0.069017 0.068269 0.067004 0.070892 0.069596 0.064077 0.064214 0.061683 M_ru_A474_ma 2 0.042992 0.040690 0.000000 0.039303 0.090482 0.091046 0.089339 0.088714 0.089202 0.090345 0.067980 0.068422 0.068102 0.067172 0.069870 0.068300 0.063604 0.063742 0.061272 M_ru_T3164_ma 3 0.043694 0.041270 0.039303 0.000000 0.088805 0.089613 0.088119 0.087525 0.087586 0.089156 0.068285 0.068452 0.068376 0.067568 0.070480 0.069550 0.064458 0.064778 0.062735 M_ru_J265_ma 4 0.088531 0.089552 0.090482 0.088805 0.000000 0.032519 0.029104 0.029393 0.030400 0.031177 0.089888 0.091214 0.090101 0.090086 0.092510 0.091214 0.093531 0.094827 0.093425 M_ru_A7875_In 5 0.089552 0.089689 0.091046 0.089613 0.032519 0.000000 0.030704 0.030354 0.027579 0.028357 0.089628 0.091290 0.090574 0.090497 0.093074 0.091687 0.094004 0.095605 0.093562 M_ru_T14456_In 6 0.087509 0.088439 0.089339 0.088119 0.029104 0.030704 0.000000 0.029104 0.028067 0.030461 0.088043 0.089766 0.088805 0.088790 0.091610 0.090375 0.092540 0.093836 0.092373 M_ru_T23245_In 7 0.086457 0.087540 0.088714 0.087525 0.029393 0.030354 0.029104 0.000000 0.029210 0.028799 0.087326 0.089385 0.088333 0.087951 0.090589 0.089263 0.091885 0.093211 0.091473 M_ru_T23246_In 8 0.086884 0.088180 0.089202 0.087586 0.030400 0.027579 0.028067 0.029210 0.000000 0.025414 0.087204 0.089202 0.088546 0.087860 0.090955 0.089385 0.091854 0.093303 0.091748 M_ru_T23416_In 9 0.089003 0.089263 0.090345 0.089156 0.031177 0.028357 0.030461 0.028799 0.025414 0.000000 0.089537 0.091321 0.090635 0.089949 0.093104 0.091290 0.093547 0.095117 0.093379 M_ru_T23478_In 10 0.063406 0.068422 0.067980 0.068285 0.089888 0.089628 0.088043 0.087326 0.087204 0.089537 0.000000 0.037733 0.039272 0.037031 0.038815 0.039654 0.063132 0.064611 0.062842 M_ru_A10311_pu 11 0.063970 0.069017 0.068422 0.068452 0.091214 0.091290 0.089766 0.089385 0.089202 0.091321 0.037733 0.000000 0.040721 0.039333 0.039684 0.040919 0.065037 0.065967 0.064138 M_ru_A10329_pu 12 0.064046 0.068269 0.068102 0.068376 0.090101 0.090574 0.088805 0.088333 0.088546 0.090635 0.039272 0.040721 0.000000 0.039898 0.041681 0.039044 0.063894 0.065495 0.064001 M_ru_A1380_pu 13 0.063025 0.067004 0.067172 0.067568 0.090086 0.090497 0.088790 0.087951 0.087860 0.089949 0.037031 0.039333 0.039898 0.000000 0.039532 0.038602 0.063299 0.064107 0.063040 M_ru_A2741_pu 14 0.065388 0.070892 0.069870 0.070480 0.092510 0.093074 0.091610 0.090589 0.090955 0.093104 0.038815 0.039684 0.041681 0.039532 0.000000 0.039684 0.066059 0.066531 0.065037 M_ru_A436_pu 15 0.064550 0.069596 0.068300 0.069550 0.091214 0.091687 0.090375 0.089263 0.089385 0.091290 0.039654 0.040919 0.039044 0.038602 0.039684 0.000000 0.064733 0.065906 0.064626 M_ru_A440_pu 16 0.052963 0.064077 0.063604 0.064458 0.093531 0.094004 0.092540 0.091885 0.091854 0.093547 0.063132 0.065037 0.063894 0.063299 0.066059 0.064733 0.000000 0.032793 0.029957 M_ru_A207_jigu 17 0.051637 0.064214 0.063742 0.064778 0.094827 0.095605 0.093836 0.093211 0.093303 0.095117 0.064611 0.065967 0.065495 0.064107 0.066531 0.065906 0.032793 0.000000 0.029119 M_ru_T3228_jigu 18 0.050905 0.061683 0.061272 0.062735 0.093425 0.093562 0.092373 0.091473 0.091